<a href="https://colab.research.google.com/github/unknownguy24/Deep-Learning/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.9.2


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist             #importing the fashion_mnist dataset from keras datasets

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()               #downloads images and labels we split into train and test

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0    #scaling our images between 0 to 1
test_images=test_images/255.0      #they are between 0 to 255 as they are greyscale images

In [ ]:
train_images[0].shape #we have got output as 28 cross 28 pixels for our first train image

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
    
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),  #creating range of integers with the name conv_1_filter where your min value is 32 and max value is 128 and step is 16
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),                 #kernel size is also called as filter size 
        activation='relu',                                                      #the value 32 and 128 tells us the number of filters in that layer
        input_shape=(28,28,1)                                                   #keras tuner will help us find out what number of filters will be good in each layer
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),   #keras tuner will go through each number of filter size values
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),                 #hp.choice chooses between 3 and 5
        activation='relu'
    ),
    keras.layers.Flatten(),                                                     #Flatten the layer
    keras.layers.Dense(                                                         #Creating the dense layer
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),    #Number of nodes/neurons in the dense layer
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')                                #Adding last dense layer with 10 o/p nodes 
    ])
  
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
    return model

In [ ]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output_',project_name="Mnist_Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9126666784286499

Best val_accuracy So Far: 0.9126666784286499
Total elapsed time: 00h 02m 45s


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 80)        800       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        46144     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 96)                3539040   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 3,586,954
Trainable params: 3,586,954
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1590 - accuracy: 0.9397 - val_loss: 0.2759 - val_accuracy: 0.9037
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1105 - accuracy: 0.9591 - val_loss: 0.2634 - val_accuracy: 0.9143
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0717 - accuracy: 0.9744 - val_loss: 0.3418 - val_accuracy: 0.9132
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0493 - accuracy: 0.9827 - val_loss: 0.4041 - val_accuracy: 0.9130
Epoch 8/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0352 - accuracy: 0.9876 - val_loss: 0.4229 - val_accuracy: 0.9143
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0272 - accuracy: 0.9907 - val_loss: 0.4475 - val_accuracy: 0.9135
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0229 - accuracy: 0.9919 - val_loss: 0.5404 - val_accurac